# IMPORT AND DRIVE SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade pip
!pip -q install numpy pandas scipy scikit-learn matplotlib seaborn joblib h5py xgboost lightgbm tensorflow

In [ ]:
import sys
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib
import seaborn
import joblib
import h5py
import xgboost
import tensorflow as tf
import lightgbm
import os,pprint
from scipy.interpolate import PchipInterpolator
from sklearn.preprocessing import StandardScaler
import warnings
import traceback

In [ ]:
print("GPU devices:", tf.config.list_physical_devices('GPU'))

GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
AE_FILE = "/content/drive/My Drive/Aurora_Prediction/AE dataset/AE dataset.txt"
MAG_INPUT_FOLDER = "/content/drive/My Drive/Aurora_Prediction/MAG dataset/"

BASE_DIR = os.path.dirname(os.path.dirname(AE_FILE))
DATA_DIR = os.path.join(BASE_DIR, "data") if os.path.exists(os.path.join(BASE_DIR, "data")) else BASE_DIR

RAW_COMBINED = os.path.join(BASE_DIR, "MAG_L2_raw_combined.csv")
FEATURES_CSV = os.path.join(BASE_DIR, "MAG_L2_features.csv")
SCALED_CSV = os.path.join(BASE_DIR, "MAG_L2_features_scaled.csv")
FINAL_CSV = os.path.join(BASE_DIR, "MAG_L2_final_features.csv")
MODEL_DATASET_CSV = os.path.join(BASE_DIR, "MAG_L2_model_dataset.csv")

SCALER_STAGE1 = os.path.join(BASE_DIR, "mag_scaler_stage1.joblib")
SCALER_FINAL = os.path.join(BASE_DIR, "mag_scaler_final.joblib")

MODELS_DIR = os.path.join(BASE_DIR, "models")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)
for sub in ['classification','regression','data_analytics']:
    os.makedirs(os.path.join(MODELS_DIR, sub), exist_ok=True)
    os.makedirs(os.path.join(PLOTS_DIR, sub), exist_ok=True)

# --- Runtime flags ---
FAST_MODE = True            # Set False for full training runs
RUN_EXTRACT = True
RUN_FEATURES = True
RUN_ADVANCED_FEATURES = True
RUN_REGRESSION = True
RUN_MERGE_AE = True
RUN_CLASSIFICATION = True
RUN_ANALYTICS = True

# --- Print confirmation ---
print("Paths and flags set — verify below:")
pprint.pprint({
    "AE_FILE": AE_FILE,
    "MAG_INPUT_FOLDER": MAG_INPUT_FOLDER,
    "BASE_DIR": BASE_DIR,
    "RAW_COMBINED": RAW_COMBINED,
    "FEATURES_CSV": FEATURES_CSV,
    "SCALED_CSV": SCALED_CSV,
    "FINAL_CSV": FINAL_CSV,
    "MODEL_DATASET_CSV": MODEL_DATASET_CSV,
    "SCALER_STAGE1": SCALER_STAGE1,
    "SCALER_FINAL": SCALER_FINAL,
    "MODELS_DIR": MODELS_DIR,
    "PLOTS_DIR": PLOTS_DIR,
    "FAST_MODE": FAST_MODE,
    "RUN_EXTRACT": RUN_EXTRACT,
    "RUN_FEATURES": RUN_FEATURES,
    "RUN_REGRESSION": RUN_REGRESSION,
    "RUN_MERGE_AE": RUN_MERGE_AE
})
print("\nChecking that the specified input paths exist:")
print("AE_FILE exists:", os.path.exists(AE_FILE))
print("MAG_INPUT_FOLDER exists:", os.path.exists(MAG_INPUT_FOLDER))
print("List MAG_INPUT_FOLDER (first 10 entries):")
if os.path.exists(MAG_INPUT_FOLDER):
    print(sorted(os.listdir(MAG_INPUT_FOLDER))[:10])
else:
    print("MAG input folder not found. Please check the path.")

Paths and flags set — verify below:
{'AE_FILE': '/content/drive/My Drive/Aurora_Prediction/AE dataset/AE '
            'dataset.txt',
 'BASE_DIR': '/content/drive/My Drive/Aurora_Prediction',
 'FAST_MODE': True,
 'FEATURES_CSV': '/content/drive/My '
                 'Drive/Aurora_Prediction/MAG_L2_features.csv',
 'FINAL_CSV': '/content/drive/My '
              'Drive/Aurora_Prediction/MAG_L2_final_features.csv',
 'MAG_INPUT_FOLDER': '/content/drive/My Drive/Aurora_Prediction/MAG dataset/',
 'MODELS_DIR': '/content/drive/My Drive/Aurora_Prediction/models',
 'MODEL_DATASET_CSV': '/content/drive/My '
                      'Drive/Aurora_Prediction/MAG_L2_model_dataset.csv',
 'PLOTS_DIR': '/content/drive/My Drive/Aurora_Prediction/plots',
 'RAW_COMBINED': '/content/drive/My '
                 'Drive/Aurora_Prediction/MAG_L2_raw_combined.csv',
 'RUN_EXTRACT': True,
 'RUN_FEATURES': True,
 'RUN_MERGE_AE': True,
 'RUN_REGRESSION': True,
 'SCALED_CSV': '/content/drive/My '
               'Drive

# PRE-PROCESSING AND FINAL CLEANUP

In [ ]:
warnings.filterwarnings('ignore')

Aurora Electrojet Time parsing

Core Helper Functions:
Implemented HDF5/.nc extraction, recursive dataset flattening, and CSV merging via combine_l2_nc_files.
Added robust time parsing (infer_time_column, parse_time_column) and PCHIP interpolation helper pchip_interpolate_series.
Included scaling helpers using StandardScaler and final_cleanup for log transforms and outlier removal.
Outputs: reusable functions used by subsequent cells to build datasets and save scaler objects.

In [ ]:
ROLL_WINDOWS = ["5min", "10min"]
LAG_STEPS = [1, 2, 3]
MIN_POINTS_FOR_INTERP = 3
EPS = 1e-6

def extract_from_group(group, prefix=""):
    """
    Recursively extract datasets from an HDF5 group and return dict{name->1D-array}.
    """
    data = {}
    for key in group.keys():
        item = group[key]
        full_name = f"{prefix}/{key}".strip("/")
        if isinstance(item, h5py.Dataset):
            try:
                arr = np.array(item[()]).flatten()
                data[full_name] = arr
            except Exception:
                pass
        elif isinstance(item, h5py.Group):
            data.update(extract_from_group(item, full_name))
    return data

def combine_l2_nc_files(input_folder, output_csv, max_files=None):
    """
    Read .nc files (prefers L2 but accepts L1), extract datasets, pad arrays and concatenate into CSV.
    Returns path to saved CSV.
    """
    all_frames = []
    processed = 0
    if not os.path.isdir(input_folder):
        raise FileNotFoundError(f"Input folder not found: {input_folder}")
    fnames = sorted(os.listdir(input_folder))
    print(f"Found {len(fnames)} files in {input_folder}")
    for fname in fnames:
        if not fname.endswith(".nc"):
            continue
        if "L2" not in fname and "L1" not in fname:
            continue
        path = os.path.join(input_folder, fname)
        try:
            with h5py.File(path, "r") as nc:
                extracted = extract_from_group(nc)
            if not extracted:
                print("  -> No readable datasets in", fname)
                continue
            max_len = max(len(v) for v in extracted.values())
            for k in list(extracted.keys()):
                arr = extracted[k]
                if len(arr) < max_len:
                    extracted[k] = np.pad(arr, (0, max_len - len(arr)), constant_values=np.nan)
            df = pd.DataFrame(extracted)
            df["source_file"] = fname
            all_frames.append(df)
            processed += 1
            print("  Processed:", fname, "rows:", df.shape[0])
            if max_files and processed >= max_files:
                break
        except Exception as e:
            print(f"  Error reading {fname}: {e}")
    if not all_frames:
        raise RuntimeError("No readable .nc files found in folder.")
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_csv(output_csv, index=False)
    print(f"Saved combined CSV ({processed} files) -> {output_csv}")
    return output_csv

def infer_time_column(df):
    """
    Heuristic to find a time column among common names or epoch-like numeric fields.
    """
    candidates = ['time', 'utc_time', 'TIME', 'Time', 'timestamp', 'epoch', 'epoch_for_cdf_mod', 'time_sec']
    for c in candidates:
        if c in df.columns:
            return c
    # fallback: choose numeric column with large values (epoch)
    for c in df.columns:
        try:
            vals = pd.to_numeric(df[c], errors='coerce').dropna()
            if not vals.empty and abs(vals.max()) > 1e6:
                return c
        except Exception:
            continue
    raise ValueError("Could not find a time column automatically.")

def parse_time_column(df, time_col):
    """
    Convert numeric epoch or string time to pandas datetime. Auto-detects units.
    """
    s = pd.to_numeric(df[time_col], errors='coerce')
    if s.dropna().empty:
        return pd.to_datetime(df[time_col], errors='coerce')
    max_val = s.dropna().abs().max()
    if max_val > 1e16:
        unit = "ns"
    elif max_val > 1e12:
        unit = "ns"
    elif max_val > 1e9:
        unit = "ms"
    elif max_val > 1e6:
        unit = "s"
    else:
        try:
            return pd.to_datetime(df[time_col], errors='coerce')
        except Exception:
            unit = "s"
    return pd.to_datetime(s, unit=unit, origin='unix', errors='coerce')

def ensure_B_columns(df):
    """
    Detect likely Bx, By, Bz column names from common variants.
    Returns mapping {'Bx': colname or None, ...}
    """
    candidates_map = {
        'Bx': ['Bx', 'Bx_gse', 'Bx_gsm', 'B_x', 'bx', 'BX'],
        'By': ['By', 'By_gse', 'By_gsm', 'B_y', 'by', 'BY'],
        'Bz': ['Bz', 'Bz_gse', 'Bz_gsm', 'B_z', 'bz', 'BZ'],
    }
    found = {}
    for k, cand in candidates_map.items():
        hit = None
        for c in cand:
            if c in df.columns:
                hit = c
                break
        found[k] = hit
    return found

def pchip_interpolate_series(time_numeric, y, target_time_numeric):
    """
    Interpolate y sampled at time_numeric to target_time_numeric with PCHIP.
    """
    mask = np.isfinite(y) & np.isfinite(time_numeric)
    if mask.sum() < MIN_POINTS_FOR_INTERP:
        return np.full_like(target_time_numeric, np.nan, dtype=float)
    try:
        f = PchipInterpolator(time_numeric[mask], y[mask], extrapolate=False)
        return f(target_time_numeric)
    except Exception:
        try:
            return np.interp(target_time_numeric, time_numeric[mask], y[mask], left=np.nan, right=np.nan)
        except Exception:
            return np.full_like(target_time_numeric, np.nan, dtype=float)

def run_feature_pipeline(input_csv, output_features_csv, scaler_path=None):
    """
    Read combined CSV, parse time, interpolate core columns with PCHIP,
    compute B_mag, dB_dt, rolling stats and lag features, save features CSV,
    and produce a stage-1 scaled CSV.
    """
    print("Loading combined CSV:", input_csv)
    raw = pd.read_csv(input_csv, dtype=str)
    time_col = infer_time_column(raw)
    print("Detected time column:", time_col)
    raw[time_col] = raw[time_col].replace('', np.nan)
    dt = parse_time_column(raw, time_col)
    raw['__datetime'] = dt
    raw = raw.dropna(subset=['__datetime']).copy()
    raw = raw.sort_values('__datetime').reset_index(drop=True)
    raw.index = pd.DatetimeIndex(raw['__datetime'])

    b_map = ensure_B_columns(raw)
    print("Detected B columns mapping:", b_map)

    # coerce numeric columns
    for c in raw.columns:
        if c == '__datetime': continue
        raw[c] = pd.to_numeric(raw[c], errors='coerce')

    df_work = raw.copy()
    time_numeric = df_work.index.view(np.int64) / 1e9  # seconds
    target_time_numeric = time_numeric

    primary_cols = [v for v in b_map.values() if v] + [c for c in ['x_gse','y_gse','z_gse','x_gsm','y_gsm','z_gsm'] if c in df_work.columns]
    primary_cols = list(dict.fromkeys(primary_cols))
    print("Columns considered for interpolation:", primary_cols)

    out = pd.DataFrame(index=df_work.index)
    out['time'] = df_work['__datetime']
    for col in primary_cols:
        y = df_work[col].to_numpy(dtype=float)
        out[col] = pchip_interpolate_series(time_numeric, y, target_time_numeric)

    # pick canonical Bx,By,Bz
    def select_first_available(choices):
        for c in choices:
            if c in out.columns and out[c].notna().any():
                return out[c].astype(float)
        return pd.Series(np.nan, index=out.index)

    out['Bx'] = select_first_available(['Bx_gse','Bx_gsm','Bx', b_map.get('Bx')])
    out['By'] = select_first_available(['By_gse','By_gsm','By', b_map.get('By')])
    out['Bz'] = select_first_available(['Bz_gse','Bz_gsm','Bz', b_map.get('Bz')])

    out['B_mag'] = np.sqrt(out['Bx']**2 + out['By']**2 + out['Bz']**2)

    dt_seconds = out.index.to_series().diff().dt.total_seconds().fillna(0).to_numpy()
    dB = out['B_mag'].diff().to_numpy()
    dt_nonzero = dt_seconds.copy()
    dt_nonzero[dt_nonzero == 0] = np.nan
    out['dB_dt'] = dB / dt_nonzero

    for rw in ROLL_WINDOWS:
        out[f'B_mag_roll_mean_{rw}'] = out['B_mag'].rolling(rw, min_periods=1).mean()
        out[f'B_mag_roll_std_{rw}'] = out['B_mag'].rolling(rw, min_periods=1).std().fillna(0)

    for lag in LAG_STEPS:
        out[f'B_mag_lag_{lag}'] = out['B_mag'].shift(lag)
        out[f'dB_dt_lag_{lag}'] = out['dB_dt'].shift(lag)

    if 'source_file' in df_work.columns:
        out['source_file'] = df_work['source_file']

    core = ['time','Bx','By','Bz','B_mag','dB_dt']
    rest = [c for c in out.columns if c not in core]
    out = out[core + rest]

    out.to_csv(output_features_csv, index=False)
    print("Saved features CSV ->", output_features_csv)

    numeric_cols = out.select_dtypes(include=[np.number]).columns.tolist()
    scaler = None
    if numeric_cols:
        scaler = StandardScaler()
        scaled = scaler.fit_transform(out[numeric_cols].fillna(0.0))
        scaled_df = pd.DataFrame(scaled, columns=numeric_cols, index=out.index)
        final_stage1 = pd.concat([out.drop(columns=numeric_cols), scaled_df], axis=1)
        stage1_output = output_features_csv.replace(".csv", "_scaled.csv")
        final_stage1.to_csv(stage1_output, index=False)
        if scaler_path:
            joblib.dump(scaler, scaler_path)
            print("Saved stage-1 scaler ->", scaler_path)
        print("Saved stage-1 scaled CSV ->", stage1_output)
        return out, final_stage1, scaler
    else:
        out.to_csv(output_features_csv, index=False)
        print("No numeric columns detected; saved features only.")
        return out, out.copy(), None

def final_cleanup(stage1_df, final_csv, final_scaler_path):
    """
    Apply log transforms, remove extreme outliers, final scaling, save final csv + scaler.
    """
    df = stage1_df.copy()
    log_features = [
        "B_mag",
        "dB_dt",
        "B_mag_roll_mean_5min",
        "B_mag_roll_std_5min",
        "B_mag_roll_mean_10min",
        "B_mag_roll_std_10min",
    ]
    for col in log_features:
        if col in df.columns:
            df[col + "_log10"] = np.log10(df[col].abs().fillna(0.0) + EPS)

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        for col in numeric_cols:
            upper = df[col].quantile(0.999)
            df = df[(df[col].isna()) | (df[col] <= upper)]

    numeric_cols_after = df.select_dtypes(include=[np.number]).columns.tolist()
    scaler_final = None
    if numeric_cols_after:
        scaler_final = StandardScaler()
        df[numeric_cols_after] = scaler_final.fit_transform(df[numeric_cols_after].fillna(0.0))
        joblib.dump(scaler_final, final_scaler_path)
        print("Saved final scaler ->", final_scaler_path)

    df.to_csv(final_csv, index=False)
    print("Saved final ML-ready CSV ->", final_csv)
    return df, scaler_final

MAG extraction

MAG .nc Extraction:
Scanned the MAG folder and recursively extracted readable datasets from .nc files, padding variable-length arrays and concatenating into MAG_L2_raw_combined.csv.
Processed a limited number of files in FAST_MODE for quick iteration, or all files when disabled.
Preserved provenance via a source_file column.
Note: large .nc files can be slow — run overnight for full extraction.

In [ ]:
print("RUN_EXTRACT:", RUN_EXTRACT)
print("MAG_INPUT_FOLDER:", MAG_INPUT_FOLDER)
print("RAW_COMBINED:", RAW_COMBINED)

if not RUN_EXTRACT:
    print("RUN_EXTRACT is False — skipping extraction.")
else:
    if not os.path.exists(MAG_INPUT_FOLDER):
        raise FileNotFoundError(f"MAG input folder not found: {MAG_INPUT_FOLDER}")
    # Use the helper function defined in Cell 4
    max_files = 20 if FAST_MODE else None
    try:
        combined_path = combine_l2_nc_files(MAG_INPUT_FOLDER, RAW_COMBINED, max_files=max_files)
        print("Combined CSV saved to:", combined_path)
        # show basic info
        import pandas as pd
        df = pd.read_csv(combined_path, nrows=5)
        print("Preview (first 5 rows):")
        print(df.head())
        print("Columns:", df.columns.tolist())
        print("Combined file size (MB):", os.path.getsize(combined_path)/1e6)
    except Exception as e:
        print("Error during extraction:", e)

RUN_EXTRACT: True
MAG_INPUT_FOLDER: /content/drive/My Drive/Aurora_Prediction/MAG dataset/
RAW_COMBINED: /content/drive/My Drive/Aurora_Prediction/MAG_L2_raw_combined.csv
Found 26 files in /content/drive/My Drive/Aurora_Prediction/MAG dataset/
  Processed: L1_MAG91N18P1AL10010309024025306040142676_N00_0000_000683_V00.nc rows: 337531
  Processed: L1_MAG91N18P1AL10010409024025307031706836_N00_0000_000684_V00.nc rows: 337463
  Processed: L1_MAG91N18P1AL10010509024025307040040402_N00_0000_000684_V00.nc rows: 337537
  Processed: L1_MAG91N18P1AL10010609024025308031532211_N00_0000_000685_V00.nc rows: 337460
  Processed: L1_MAG91N18P1AL10010709024025308035956252_N00_0000_000685_V00.nc rows: 337533
  Processed: L1_MAG91N18P1AL10010809024025309031517596_N00_0000_000686_V00.nc rows: 337466
  Processed: L1_MAG91N18P1AL10010909024025309035900499_N00_0000_000686_V00.nc rows: 337532
  Processed: L1_MAG91N18P1AL10011009024025310031431084_N00_0000_000687_V00.nc rows: 337469
  Processed: L1_MAG91N18P1AL

# FEATURE EXTRACTION STAGE(1)

Feature Engineering (Stage 1):
Ran run_feature_pipeline to parse time, interpolate core fields (Bx/By/Bz), compute B_mag and dB_dt, add rolling stats and lag features, and saved MAG_L2_features.csv.
Produced a stage-1 scaled CSV (numeric cols standardized) and saved the stage-1 scaler as mag_scaler_stage1.joblib.
This creates a consistent, time-indexed features table ready for final cleanup.
Note: PCHIP interpolation preserves signal shape for magnetometer data.

In [ ]:
print("INPUT combined CSV expected at:", RAW_COMBINED)
if not os.path.exists(RAW_COMBINED):
    raise FileNotFoundError(f"Combined raw CSV not found at {RAW_COMBINED}. Run extraction (Cell 5) or set correct path.")

try:
    # run_feature_pipeline is defined in Cell 4
    features_unscaled, features_stage1_df, scaler_stage1 = run_feature_pipeline(RAW_COMBINED, FEATURES_CSV, SCALER_STAGE1)
    print("Feature pipeline completed.")
    print("Unscaled features preview:")
    display(features_unscaled.head())
    print("Stage-1 scaled preview:")
    display(features_stage1_df.head())
    if scaler_stage1 is not None:
        print("Stage-1 scaler saved to:", SCALER_STAGE1)
    # Also save a simple CSV copy to MODEL_DATASET_CSV placeholder (will be updated after final cleanup)
    features_stage1_df.to_csv(MODEL_DATASET_CSV.replace(".csv","_stage1.csv"), index=False)
    print("Stage-1 dataset saved to:", MODEL_DATASET_CSV.replace(".csv","_stage1.csv"))
except Exception as e:
    print("Feature engineering failed:")
    traceback.print_exc()

INPUT combined CSV expected at: /content/drive/My Drive/Aurora_Prediction/MAG_L2_raw_combined.csv
Loading combined CSV: /content/drive/My Drive/Aurora_Prediction/MAG_L2_raw_combined.csv
Detected time column: time
Detected B columns mapping: {'Bx': 'Bx', 'By': 'By', 'Bz': 'Bz'}
Columns considered for interpolation: ['Bx', 'By', 'Bz']
Saved features CSV -> /content/drive/My Drive/Aurora_Prediction/MAG_L2_features.csv
Saved stage-1 scaler -> /content/drive/My Drive/Aurora_Prediction/mag_scaler_stage1.joblib
Saved stage-1 scaled CSV -> /content/drive/My Drive/Aurora_Prediction/MAG_L2_features_scaled.csv
Feature pipeline completed.
Unscaled features preview:


,time,Bx,By,Bz,B_mag,dB_dt,B_mag_roll_mean_5min,B_mag_roll_std_5min,B_mag_roll_mean_10min,B_mag_roll_std_10min,B_mag_lag_1,dB_dt_lag_1,B_mag_lag_2,dB_dt_lag_2,B_mag_lag_3,dB_dt_lag_3,source_file
__datetime,,,,,,,,,,,,,,,,,
1970-01-21 09:26:38.401597295,1970-01-21 09:26:38.401597295,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-21 09:26:38.401725295,1970-01-21 09:26:38.401725295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-21 09:26:38.401853295,1970-01-21 09:26:38.401853295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1970-01-21 09:26:38.401981295,1970-01-21 09:26:38.401981295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1970-01-21 09:26:38.402109295,1970-01-21 09:26:38.402109295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


Stage-1 scaled preview:


,time,Bx,By,Bz,B_mag,dB_dt,B_mag_roll_mean_5min,B_mag_roll_std_5min,B_mag_roll_mean_10min,B_mag_roll_std_10min,B_mag_lag_1,dB_dt_lag_1,B_mag_lag_2,dB_dt_lag_2,B_mag_lag_3,dB_dt_lag_3,source_file
__datetime,,,,,,,,,,,,,,,,,
1970-01-21 09:26:38.401597295,1970-01-21 09:26:38.401597295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401725295,1970-01-21 09:26:38.401725295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401853295,1970-01-21 09:26:38.401853295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401981295,1970-01-21 09:26:38.401981295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.402109295,1970-01-21 09:26:38.402109295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Stage-1 scaler saved to: /content/drive/My Drive/Aurora_Prediction/mag_scaler_stage1.joblib
Stage-1 dataset saved to: /content/drive/My Drive/Aurora_Prediction/MAG_L2_model_dataset_stage1.csv


Final Cleanup (Stage 2):
Applied log10 transforms to selected features, removed extreme outliers (>99.9 percentile), and ran a second StandardScaler to produce MAG_L2_final_features.csv.
Saved the final scaler as mag_scaler_final.joblib and exported MAG_L2_model_dataset.csv for modeling.
Result: ML-ready dataset with stabilized distributions and reproducible scaling.
Tip: keep scalers with your models for inference consistency.

In [ ]:
print("Looking for features_unscaled in memory or FEATURES_CSV on disk...")
if 'features_unscaled' in globals():
    stage1_unscaled = features_unscaled
    print("Using `features_unscaled` from memory.")
elif os.path.exists(FEATURES_CSV):
    stage1_unscaled = pd.read_csv(FEATURES_CSV, parse_dates=['time'], infer_datetime_format=True)
    print("Loaded FEATURES_CSV from disk:", FEATURES_CSV)
else:
    raise FileNotFoundError("No `features_unscaled` in memory and FEATURES_CSV missing. Run Cell 6 first.")

try:
    final_df, scaler_final = final_cleanup(stage1_unscaled, FINAL_CSV, SCALER_FINAL)
    # Also save as MODEL_DATASET_CSV for downstream cells
    final_df.to_csv(MODEL_DATASET_CSV, index=False)
    print("Final ML-ready dataset saved to:", FINAL_CSV)
    print("Also copied to MODEL_DATASET_CSV:", MODEL_DATASET_CSV)
    if scaler_final is not None:
        print("Final scaler saved to:", SCALER_FINAL)
    print("\nFinal dataset shape:", final_df.shape)
    display(final_df.head())
except Exception as e:
    print("Final cleanup failed:")
    traceback.print_exc()

Looking for features_unscaled in memory or FEATURES_CSV on disk...
Using `features_unscaled` from memory.
Saved final scaler -> /content/drive/My Drive/Aurora_Prediction/mag_scaler_final.joblib
Saved final ML-ready CSV -> /content/drive/My Drive/Aurora_Prediction/MAG_L2_final_features.csv
Final ML-ready dataset saved to: /content/drive/My Drive/Aurora_Prediction/MAG_L2_final_features.csv
Also copied to MODEL_DATASET_CSV: /content/drive/My Drive/Aurora_Prediction/MAG_L2_model_dataset.csv
Final scaler saved to: /content/drive/My Drive/Aurora_Prediction/mag_scaler_final.joblib

Final dataset shape: (6749964, 23)


,time,Bx,By,Bz,B_mag,dB_dt,B_mag_roll_mean_5min,B_mag_roll_std_5min,B_mag_roll_mean_10min,B_mag_roll_std_10min,...,dB_dt_lag_2,B_mag_lag_3,dB_dt_lag_3,source_file,B_mag_log10,dB_dt_log10,B_mag_roll_mean_5min_log10,B_mag_roll_std_5min_log10,B_mag_roll_mean_10min_log10,B_mag_roll_std_10min_log10
__datetime,,,,,,,,,,,,,,,,,,,,,
1970-01-21 09:26:38.401597295,1970-01-21 09:26:38.401597295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401725295,1970-01-21 09:26:38.401725295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401853295,1970-01-21 09:26:38.401853295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.401981295,1970-01-21 09:26:38.401981295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970-01-21 09:26:38.402109295,1970-01-21 09:26:38.402109295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Advanced Spike Cleaning & Recompute Features:
Implemented rolling-MAD spike detection + local-median replacement and a 5-point median filter on Bx/By/Bz to remove high-frequency sensor spikes.
Recomputed B_mag, dB_dt, rolling means/stds, lag features, and log transforms, then re-scaled selected columns and saved the cleaned MAG_L2_model_dataset.csv.
Saved an additional scaler mag_spike_clean_scaler.joblib for the cleaned components.
Note: cleaning reduces false dB/dt spikes that hurt model training.

In [ ]:
# Cell 8: Advanced spike cleaning, recompute B_mag/dB_dt, rolling features, lags, and scale
import pandas as pd, numpy as np, os, traceback
from scipy.stats import median_abs_deviation
from scipy.signal import medfilt
from sklearn.preprocessing import StandardScaler
import joblib

print("Loading model dataset:", MODEL_DATASET_CSV)
if not os.path.exists(MODEL_DATASET_CSV):
    raise FileNotFoundError(f"MODEL_DATASET_CSV not found: {MODEL_DATASET_CSV}. Run previous cells first.")

# Load (this should be the final CSV from cleanup)
df = pd.read_csv(MODEL_DATASET_CSV, parse_dates=['time'], infer_datetime_format=True, low_memory=False)
print("Loaded shape:", df.shape)

# Ensure time present
if 'time' not in df.columns:
    # If no `time` column, try to detect datetime-like column
    for c in df.columns:
        if np.issubdtype(df[c].dtype, np.datetime64):
            df = df.rename(columns={c: 'time'})
            break
    if 'time' not in df.columns:
        # create index-based datetime as fallback
        df['time'] = pd.date_range(start='1970-01-01', periods=len(df), freq='S')

df = df.sort_values('time').reset_index(drop=True)

# Spike cleaning function (rolling MAD + median filter)
def clean_series(series, window_size=15, n_sigmas=3):
    s = series.copy().astype(float).fillna(0.0).to_numpy()
    cleaned = s.copy()
    L = window_size // 2
    k = 1.4826  # MAD -> std approx
    N = len(s)
    # avoid extremes at edges by leaving first/last L points unchanged except median filter after
    for i in range(L, N - L):
        window = s[i-L:i+L+1]
        med = np.median(window)
        mad = median_abs_deviation(window, scale='normal')  # returns scaled MAD by default? ensure fallback below
        # robust fallback:
        if np.isnan(mad) or mad == 0:
            mad = np.median(np.abs(window - med)) or 1e-6
        threshold = n_sigmas * k * mad
        if abs(s[i] - med) > threshold:
            cleaned[i] = med
    # apply small median filter at the end
    try:
        cleaned = medfilt(cleaned, kernel_size=5)
    except Exception:
        pass
    return pd.Series(cleaned, index=series.index)

# Apply cleaning to Bx/By/Bz if present
for comp in ['Bx','By','Bz']:
    if comp in df.columns:
        print("Cleaning:", comp)
        df[f'{comp}_clean'] = clean_series(pd.to_numeric(df[comp], errors='coerce').fillna(0.0))
    else:
        print("Column missing, skipping:", comp)

# Recompute B_mag from cleaned components (prefer cleaned if available)
if all(c in df.columns for c in ['Bx_clean','By_clean','Bz_clean']):
    df['B_mag'] = np.sqrt(df['Bx_clean']**2 + df['By_clean']**2 + df['Bz_clean']**2)
else:
    # fallback to raw columns if cleaned not available
    df['B_mag'] = np.sqrt(df.get('Bx',0)**2 + df.get('By',0)**2 + df.get('Bz',0)**2)

# Compute dB/dt as first difference (preserving time delta in seconds if time index is datetime)
df['dB_dt'] = df['B_mag'].diff()
if np.issubdtype(df['time'].dtype, np.datetime64):
    dt_seconds = df['time'].diff().dt.total_seconds().fillna(0)
    dt_nonzero = dt_seconds.replace(0, np.nan)
    df['dB_dt'] = df['B_mag'].diff() / dt_nonzero
else:
    # sample-based diff (fallback)
    df['dB_dt'] = df['B_mag'].diff()

# Interpolate small gaps in numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[num_cols] = df[num_cols].interpolate(method='linear').fillna(method='bfill').fillna(method='ffill')

# Rolling statistics (use sample windows 5 and 10)
for w in [5, 10]:
    df[f'B_mag_mean_{w}'] = df['B_mag'].rolling(window=w, min_periods=1).mean()
    df[f'B_mag_std_{w}'] = df['B_mag'].rolling(window=w, min_periods=1).std().fillna(0.0)

# Lag features
for lag in [1,2,3]:
    df[f'B_mag_lag_{lag}'] = df['B_mag'].shift(lag)
    df[f'dB_dt_lag_{lag}'] = df['dB_dt'].shift(lag)

# Log transforms
df['B_mag_log10'] = np.log10(np.abs(df['B_mag'].fillna(0.0)) + 1e-12)
df['dB_dt_log10'] = np.log10(np.abs(df['dB_dt'].fillna(0.0)) + 1e-12)

# Drop rows with NaNs introduced by lagging
df = df.dropna().reset_index(drop=True)
print("After cleaning/features shape:", df.shape)

# Scale selected columns for modeling (keep scaler)
scale_cols = [c for c in ['Bx_clean','By_clean','Bz_clean','B_mag','dB_dt'] if c in df.columns]
scaler = None
if scale_cols:
    scaler = StandardScaler()
    df[scale_cols] = scaler.fit_transform(df[scale_cols])
    # save scaler
    scaler_path = os.path.join(MODELS_DIR, "scalers")
    os.makedirs(scaler_path, exist_ok=True)
    joblib.dump(scaler, os.path.join(scaler_path, "mag_spike_clean_scaler.joblib"))
    print("Saved spike-clean scaler to:", os.path.join(scaler_path, "mag_spike_clean_scaler.joblib"))

# Save processed dataset for modeling
df.to_csv(MODEL_DATASET_CSV, index=False)
print("Saved cleaned model dataset to:", MODEL_DATASET_CSV)
print("Preview:")
display(df.head())

# If FAST_MODE, optionally write a small sample for fast prototyping
if FAST_MODE:
    sample_path = MODEL_DATASET_CSV.replace(".csv","_fast_sample.csv")
    df.head(5000).to_csv(sample_path, index=False)
    print("Saved FAST sample to:", sample_path)

# Done
print("Advanced spike cleaning completed.")

Loading model dataset: /content/drive/My Drive/Aurora_Prediction/MAG_L2_model_dataset.csv
Loaded shape: (6749964, 23)
Cleaning: Bx


KeyboardInterrupt: 

# REGRESSION TRAINING

Regression Training (RF, XGBoost, LSTM):
Built target B_mag_target = B_mag(t+1), split timewise (80/20), trained Random Forest and XGBoost regressors, and saved them under regression.
Prepared sequences and trained an LSTM (when TensorFlow and enough data present), with early stopping and reduced epochs in FAST_MODE.
Saved models: random_forest_model.joblib, xgboost_model.joblib, and lstm_model.keras.
Outputs: model comparison CSV with RMSE/MAE/R² for quick model selection.

In [ ]:
# Cell 9: Regression pipeline - RandomForest, XGBoost, LSTM
import os, numpy as np, pandas as pd, joblib, traceback
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

print("Loading model dataset:", MODEL_DATASET_CSV)
if not os.path.exists(MODEL_DATASET_CSV):
    raise FileNotFoundError(f"MODEL_DATASET_CSV not found: {MODEL_DATASET_CSV}. Run previous cells first.")

df = pd.read_csv(MODEL_DATASET_CSV, low_memory=False)
print("Dataset shape:", df.shape)
# Create target: B_mag at t+1
if 'B_mag' not in df.columns:
    raise KeyError("B_mag column not found in MODEL_DATASET_CSV")

df['B_mag_target'] = df['B_mag'].shift(-1)
df = df.dropna().reset_index(drop=True)
print("After creating target, shape:", df.shape)

# Select numeric features excluding time/source/target
exclude_cols = ['time','B_mag_target','source_file'] if 'time' in df.columns else ['B_mag_target','source_file']
feature_cols = [c for c in df.columns if c not in exclude_cols and np.issubdtype(df[c].dtype, np.number)]
print(f"Using {len(feature_cols)} features for regression.")

X = df[feature_cols].values
y = df['B_mag_target'].values

# Train/test split (time-series)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# ---------------- Random Forest (baseline) ----------------
print("\nTraining Random Forest...")
rf = RandomForestRegressor(
    n_estimators=100 if FAST_MODE else 300,
    max_depth=20 if FAST_MODE else 30,
    min_samples_leaf=4, random_state=42, n_jobs=-1
)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)
print(f"RF - RMSE: {rf_rmse:.4f}, MAE: {rf_mae:.4f}, R2: {rf_r2:.4f}")
joblib.dump(rf, os.path.join(MODELS_DIR, 'regression', 'random_forest_model.joblib'))
print("Saved RF model.")

# ---------------- XGBoost ----------------
print("\nTraining XGBoost...")
xgb = XGBRegressor(
    n_estimators=100 if FAST_MODE else 300,
    max_depth=6 if FAST_MODE else 10,
    learning_rate=0.1,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, n_jobs=-1, verbosity=0
)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)
print(f"XGB - RMSE: {xgb_rmse:.4f}, MAE: {xgb_mae:.4f}, R2: {xgb_r2:.4f}")
joblib.dump(xgb, os.path.join(MODELS_DIR, 'regression', 'xgboost_model.joblib'))
print("Saved XGBoost model.")

# ---------------- LSTM ----------------
print("\nPreparing sequences for LSTM...")
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
except Exception as e:
    tf = None
    print("TensorFlow import failed:", e)

SEQ_LEN = 10
def create_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, SEQ_LEN)
print("LSTM shapes:", X_train_seq.shape, X_test_seq.shape)

if tf is not None and X_train_seq.size > 0:
    print("\nBuilding LSTM model...")
    lstm = keras.Sequential([
        layers.LSTM(64, activation='tanh', return_sequences=True, input_shape=(SEQ_LEN, X_train.shape[1])),
        layers.Dropout(0.2),
        layers.LSTM(32, activation='tanh'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    lstm.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3 if FAST_MODE else 10, restore_best_weights=True)
    history = lstm.fit(X_train_seq, y_train_seq, validation_split=0.2, epochs=5 if FAST_MODE else 50, batch_size=32, callbacks=[early], verbose=1)
    lstm_pred = lstm.predict(X_test_seq).flatten()
    lstm_rmse = np.sqrt(mean_squared_error(y_test_seq, lstm_pred))
    lstm_mae = mean_absolute_error(y_test_seq, lstm_pred)
    lstm_r2 = r2_score(y_test_seq, lstm_pred)
    print(f"LSTM - RMSE: {lstm_rmse:.4f}, MAE: {lstm_mae:.4f}, R2: {lstm_r2:.4f}")
    lstm.save(os.path.join(MODELS_DIR, 'regression', 'lstm_model.keras'))
    print("Saved LSTM model.")
else:
    print("Skipping LSTM training: TensorFlow missing or insufficient data.")
    lstm_rmse = lstm_mae = lstm_r2 = None

# ---------------- Summary ----------------
results = pd.DataFrame({
    'Model': ['RandomForest','XGBoost','LSTM'],
    'RMSE': [rf_rmse, xgb_rmse, lstm_rmse],
    'MAE':  [rf_mae, xgb_mae, lstm_mae],
    'R2':   [rf_r2, xgb_r2, lstm_r2]
})
out_csv = os.path.join(MODELS_DIR, 'regression', 'model_comparison.csv')
results.to_csv(out_csv, index=False)
print("\nModel comparison saved to:", out_csv)
print(results)

Auroral electrojet merging and training

AE Merge & Labeling:
Correctly parsed AE_dataset.txt (handles zero-padded HHMMSS), computed averaged AE = mean(AE1..AE4), built datetime, and aligned AE to MAG via pd.merge_asof with a 3‑minute tolerance.
Created binary labels aurora_label = (AE > 20) and saved Aurora_Classification_dataset.csv, Aurora_Train.csv, and Aurora_Test.csv.
Ensured stratified train/test split where possible to preserve class balance.
Important fix: HHMMSS parsing avoids hour/minute/second misalignment that would corrupt labels.

In [ ]:
# Cell 10: AE merge & labeling - parse AE file, align with MAG, create labels and train/test CSVs
import os, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split

print("AE file:", AE_FILE)
print("Model dataset (MAG):", MODEL_DATASET_CSV)

# --- 1) load MAG dataset ---
if not os.path.exists(MODEL_DATASET_CSV):
    raise FileNotFoundError(f"Model dataset not found: {MODEL_DATASET_CSV}. Run previous cells first.")
mag = pd.read_csv(MODEL_DATASET_CSV, low_memory=False)
# ensure datetime column
if 'time' in mag.columns:
    mag['datetime'] = pd.to_datetime(mag['time'], errors='coerce')
else:
    # attempt to find a datetime-like column
    for c in mag.columns:
        if np.issubdtype(mag[c].dtype, np.datetime64):
            mag['datetime'] = mag[c]
            break
    if 'datetime' not in mag.columns:
        mag['datetime'] = pd.date_range('1970-01-01', periods=len(mag), freq='S')
mag = mag.sort_values('datetime').reset_index(drop=True)
print("MAG records:", len(mag))

# --- 2) load AE dataset with correct HHMMSS parsing ---
if not os.path.exists(AE_FILE):
    raise FileNotFoundError(f"AE file not found: {AE_FILE}")

# AE file has no header, whitespace-separated. Columns: year month day hhmmss AE1 AE2 AE3 AE4
ae = pd.read_csv(AE_FILE, sep=r'\s+', header=None, names=['year','month','day','hhmmss','AE1','AE2','AE3','AE4'], dtype={'hhmmss':str})
# normalize hhmmss strings (pad to length 6)
ae['hhmmss'] = ae['hhmmss'].str.zfill(6)

# extract hour, minute, second
ae['hour'] = ae['hhmmss'].str.slice(0,2).astype(int)
ae['minute'] = ae['hhmmss'].str.slice(2,4).astype(int)
ae['second'] = ae['hhmmss'].str.slice(4,6).astype(int)

ae['datetime'] = pd.to_datetime(ae[['year','month','day','hour','minute','second']], errors='coerce')
ae = ae.sort_values('datetime').reset_index(drop=True)
print("AE records:", len(ae))
print("AE datetime range:", ae['datetime'].min(), "->", ae['datetime'].max())

# --- 3) compute AE average and basic QC ---
ae['AE'] = ae[['AE1','AE2','AE3','AE4']].apply(pd.to_numeric, errors='coerce').mean(axis=1)
print("AE stats:")
print(ae['AE'].describe())

# Optional: drop NA datetimes
ae = ae.dropna(subset=['datetime']).reset_index(drop=True)

# --- 4) align AE to MAG using merge_asof (nearest) ---
# ensure both sorted
mag_sorted = mag.sort_values('datetime').reset_index(drop=True)
ae_sorted = ae[['datetime','AE']].sort_values('datetime').reset_index(drop=True)

# choose a tolerance: since AE sampling ~150s (2.5min), allow 3 minutes
tolerance = pd.Timedelta('3min')
merged = pd.merge_asof(mag_sorted, ae_sorted, on='datetime', direction='nearest', tolerance=tolerance)

# drop rows where AE could not be matched
matched = merged.dropna(subset=['AE']).reset_index(drop=True)
print("Aligned records (within tolerance):", len(matched), "of", len(mag_sorted))

# --- 5) create binary aurora label ---
# threshold selection: default 20 (from project); you can adjust or compute percentile
AE_THRESHOLD = 20
matched['aurora_label'] = (matched['AE'] > AE_THRESHOLD).astype(int)
print("Aurora label distribution:")
print(matched['aurora_label'].value_counts())

# --- 6) save full dataset & train/test split ---
OUT_FULL = os.path.join(BASE_DIR, 'Aurora_Classification_dataset.csv')
OUT_TRAIN = os.path.join(BASE_DIR, 'Aurora_Train.csv')
OUT_TEST  = os.path.join(BASE_DIR, 'Aurora_Test.csv')

matched.to_csv(OUT_FULL, index=False)
print("Saved full classification dataset to:", OUT_FULL)

# Stratified split
feature_cols = [c for c in matched.columns if c not in ['time','datetime','aurora_label','AE','source_file']]
X = matched[feature_cols]
y = matched['aurora_label']

# If not enough positive samples, adjust stratify parameter
stratify = y if y.nunique() > 1 and y.sum() > 5 else None
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=stratify)

train_data = matched.loc[X_train.index]
test_data  = matched.loc[X_test.index]

train_data.to_csv(OUT_TRAIN, index=False)
test_data.to_csv(OUT_TEST, index=False)
print("Saved train/test CSVs:")
print(" -", OUT_TRAIN, "(", len(train_data), "rows )")
print(" -", OUT_TEST, "(", len(test_data), "rows )")

# quick class balance check
print("\nTrain class balance:", train_data['aurora_label'].value_counts(normalize=True).to_dict())
print("Test class balance:", test_data['aurora_label'].value_counts(normalize=True).to_dict())

# done
print("\nAE merge & labeling complete. If class imbalance is extreme, consider adjusting AE_THRESHOLD or using different labeling strategy.")

Classification training (model with best fit)

In [ ]:
# Cell 11: Classification training - use regression meta-features + train classifiers
import os, joblib, pandas as pd, numpy as np, traceback
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, average_precision_score, f1_score, accuracy_score, matthews_corrcoef

print("Loading train/test CSVs...")
TRAIN_PATH = os.path.join(BASE_DIR, 'Aurora_Train.csv')
TEST_PATH  = os.path.join(BASE_DIR, 'Aurora_Test.csv')
if not os.path.exists(TRAIN_PATH) or not os.path.exists(TEST_PATH):
    raise FileNotFoundError("Train/Test CSVs not found. Run AE merge cell (Cell 10).")

train_df = pd.read_csv(TRAIN_PATH, low_memory=False)
test_df  = pd.read_csv(TEST_PATH, low_memory=False)
print("Train rows:", len(train_df), "Test rows:", len(test_df))

# Features to exclude
exclude_cols = ['datetime','aurora_label','source_file','time','AE']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]
print("Number of base features:", len(feature_cols))

X_train_raw = train_df[feature_cols].fillna(0)
X_test_raw  = test_df[feature_cols].fillna(0)
y_train = train_df['aurora_label']
y_test  = test_df['aurora_label']

# ---------------- Create meta-features from regression models ----------------
print("\nAttempting to load regression models for meta-features...")
reg_models_dir = os.path.join(MODELS_DIR, 'regression')
rf_model_path = os.path.join(reg_models_dir, 'random_forest_model.joblib')
xgb_model_path = os.path.join(reg_models_dir, 'xgboost_model.joblib')
lstm_model_path = os.path.join(reg_models_dir, 'lstm_model.keras')

models = {'rf': None, 'xgb': None, 'lstm': None}
try:
    if os.path.exists(rf_model_path):
        models['rf'] = joblib.load(rf_model_path)
        print("Loaded RF regressor.")
    if os.path.exists(xgb_model_path):
        models['xgb'] = joblib.load(xgb_model_path)
        print("Loaded XGB regressor.")
    if os.path.exists(lstm_model_path):
        try:
            from tensorflow import keras
            models['lstm'] = keras.models.load_model(lstm_model_path)
            print("Loaded LSTM regressor.")
        except Exception as e:
            print("LSTM load failed:", e)
except Exception as e:
    print("Regression model load error:", e)

# Align sequences for LSTM meta-feature if possible
SEQ_LEN = 10
def create_sequences(X, seq_len):
    X_seq = []
    for i in range(len(X) - seq_len + 1):
        X_seq.append(X[i:i+seq_len])
    return np.array(X_seq)

X_train_aligned = X_train_raw.copy().reset_index(drop=True)
X_test_aligned  = X_test_raw.copy().reset_index(drop=True)

# Add RF/XGB predictions as meta-features if models available
if models['rf'] is not None:
    try:
        X_train_aligned['rf_pred'] = models['rf'].predict(X_train_raw.values)
        X_test_aligned['rf_pred']  = models['rf'].predict(X_test_raw.values)
        print("Added rf_pred meta-feature.")
    except Exception as e:
        print("Failed to generate rf_pred:", e)

if models['xgb'] is not None:
    try:
        X_train_aligned['xgb_pred'] = models['xgb'].predict(X_train_raw.values)
        X_test_aligned['xgb_pred']  = models['xgb'].predict(X_test_raw.values)
        print("Added xgb_pred meta-feature.")
    except Exception as e:
        print("Failed to generate xgb_pred:", e)

# LSTM meta-feature: ensure sequence lengths align
if models['lstm'] is not None:
    try:
        X_train_seq = create_sequences(X_train_raw.values, SEQ_LEN)
        X_test_seq  = create_sequences(X_test_raw.values, SEQ_LEN)
        # predictions length will be len(X_seq); align by trimming aligned frames
        train_preds = models['lstm'].predict(X_train_seq, verbose=0).flatten()
        test_preds  = models['lstm'].predict(X_test_seq, verbose=0).flatten()
        # align by trimming first SEQ_LEN-1 rows
        X_train_aligned = X_train_aligned.iloc[SEQ_LEN-1:].reset_index(drop=True)
        X_test_aligned  = X_test_aligned.iloc[SEQ_LEN-1:].reset_index(drop=True)
        y_train_aligned = y_train.iloc[SEQ_LEN-1:].reset_index(drop=True)
        y_test_aligned  = y_test.iloc[SEQ_LEN-1:].reset_index(drop=True)
        X_train_aligned['lstm_pred'] = train_preds
        X_test_aligned['lstm_pred'] = test_preds
        print("Added lstm_pred meta-feature and aligned sequences.")
    except Exception as e:
        print("Failed to add lstm_pred:", e)
        # fallback to non-sequence alignment
        y_train_aligned = y_train.copy()
        y_test_aligned = y_test.copy()
else:
    y_train_aligned = y_train.copy()
    y_test_aligned = y_test.copy()

# If LSTM not used, ensure indexes align
if 'lstm_pred' not in X_train_aligned.columns:
    # make sure y_train_aligned/y_test_aligned set
    y_train_aligned = y_train.copy()
    y_test_aligned = y_test.copy()

print("Final train samples:", len(X_train_aligned), "Final test samples:", len(X_test_aligned))

# ---------------- Train classifiers ----------------
print("\nTraining classifiers...")

# Standardize for logistic regression
scaler = StandardScaler()
Xtr_scaled = scaler.fit_transform(X_train_aligned)
Xte_scaled = scaler.transform(X_test_aligned)

out_dir = os.path.join(MODELS_DIR, 'classification')
os.makedirs(out_dir, exist_ok=True)

results = {}

# Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=200 if not FAST_MODE else 100, max_depth=25, class_weight='balanced', random_state=42, n_jobs=-1)
rf_clf.fit(X_train_aligned, y_train_aligned)
y_pred_rf = rf_clf.predict(X_test_aligned)
y_proba_rf = rf_clf.predict_proba(X_test_aligned)[:,1]
results['RandomForest'] = {
    'accuracy': accuracy_score(y_test_aligned, y_pred_rf),
    'f1_score': f1_score(y_test_aligned, y_pred_rf),
    'roc_auc': roc_auc_score(y_test_aligned, y_proba_rf),
    'avg_precision': average_precision_score(y_test_aligned, y_proba_rf),
    'mcc': matthews_corrcoef(y_test_aligned, y_pred_rf),
    'y_pred': y_pred_rf,
    'y_proba': y_proba_rf
}
joblib.dump(rf_clf, os.path.join(out_dir, 'random_forest.joblib'))
print("Trained & saved RandomForest classifier.")

# XGBoost classifier
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(n_estimators=200 if not FAST_MODE else 100, max_depth=8, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss', n_jobs=-1, random_state=42)
xgb_clf.fit(X_train_aligned, y_train_aligned)
y_pred_xgb = xgb_clf.predict(X_test_aligned)
y_proba_xgb = xgb_clf.predict_proba(X_test_aligned)[:,1]
results['XGBoost'] = {
    'accuracy': accuracy_score(y_test_aligned, y_pred_xgb),
    'f1_score': f1_score(y_test_aligned, y_pred_xgb),
    'roc_auc': roc_auc_score(y_test_aligned, y_proba_xgb),
    'avg_precision': average_precision_score(y_test_aligned, y_proba_xgb),
    'mcc': matthews_corrcoef(y_test_aligned, y_pred_xgb),
    'y_pred': y_pred_xgb,
    'y_proba': y_proba_xgb
}
joblib.dump(xgb_clf, os.path.join(out_dir, 'xgboost.joblib'))
print("Trained & saved XGBoost classifier.")

# LightGBM (optional)
try:
    import lightgbm as lgb
    lgb_clf = lgb.LGBMClassifier(n_estimators=200 if not FAST_MODE else 100, max_depth=15, class_weight='balanced', random_state=42, n_jobs=-1)
    lgb_clf.fit(X_train_aligned, y_train_aligned)
    y_pred_lgb = lgb_clf.predict(X_test_aligned)
    y_proba_lgb = lgb_clf.predict_proba(X_test_aligned)[:,1]
    results['LightGBM'] = {
        'accuracy': accuracy_score(y_test_aligned, y_pred_lgb),
        'f1_score': f1_score(y_test_aligned, y_pred_lgb),
        'roc_auc': roc_auc_score(y_test_aligned, y_proba_lgb),
        'avg_precision': average_precision_score(y_test_aligned, y_proba_lgb),
        'mcc': matthews_corrcoef(y_test_aligned, y_pred_lgb),
        'y_pred': y_pred_lgb,
        'y_proba': y_proba_lgb
    }
    joblib.dump(lgb_clf, os.path.join(out_dir, 'lightgbm.joblib'))
    print("Trained & saved LightGBM classifier.")
except Exception as e:
    print("LightGBM not available or failed:", e)

# Logistic Regression
lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(Xtr_scaled, y_train_aligned)
y_pred_lr = lr.predict(Xte_scaled)
# get probabilities using scaler applied test set
y_proba_lr = lr.predict_proba(Xte_scaled)[:,1]
results['LogisticRegression'] = {
    'accuracy': accuracy_score(y_test_aligned, y_pred_lr),
    'f1_score': f1_score(y_test_aligned, y_pred_lr),
    'roc_auc': roc_auc_score(y_test_aligned, y_proba_lr),
    'avg_precision': average_precision_score(y_test_aligned, y_proba_lr),
    'mcc': matthews_corrcoef(y_test_aligned, y_pred_lr),
    'y_pred': y_pred_lr,
    'y_proba': y_proba_lr
}
joblib.dump(lr, os.path.join(out_dir, 'logistic_regression_model.joblib'))
joblib.dump(scaler, os.path.join(out_dir, 'logistic_regression_scaler.joblib'))
print("Trained & saved Logistic Regression.")

# ---------------- Save results summary ----------------
summary = pd.DataFrame({name: {k:v for k,v in vals.items() if k not in ['y_pred','y_proba']} for name,vals in results.items()}).T
summary.to_csv(os.path.join(out_dir, 'model_comparison_results.csv'))
print("\nClassification results summary:")
print(summary)

# Save best model info
best_model = summary['roc_auc'].idxmax()
with open(os.path.join(out_dir, 'best_model_info.txt'),'w') as f:
    f.write(f"Best Model: {best_model}\nROC-AUC: {summary.loc[best_model,'roc_auc']:.4f}\n")
print("Saved models to:", out_dir)

# DATA ANALYTICS AND FINAL PLOTTING

In [ ]:
# Cell 12: Data analytics & plotting - dataset overview, distributions, correlations, PCA/t-SNE, pairplots
import os, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
OUTPUT_DIR = os.path.join(PLOTS_DIR, 'data_analytics')
os.makedirs(OUTPUT_DIR, exist_ok=True)

TRAIN_PATH = os.path.join(BASE_DIR, 'Aurora_Train.csv')
TEST_PATH  = os.path.join(BASE_DIR, 'Aurora_Test.csv')

print("Loading:", TRAIN_PATH, TEST_PATH)
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)
print("Train shape:", train_df.shape, "Test shape:", test_df.shape)

# Identify feature columns
exclude_cols = ['aurora_label','datetime','source_file','time','AE']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]
FEATURES_TO_PLOT = feature_cols[:8] if len(feature_cols) > 8 else feature_cols

# 1) Dataset overview: class distribution & missing
fig, axes = plt.subplots(2,2, figsize=(14,8))
ax = axes[0,0]
counts = train_df['aurora_label'].value_counts()
ax.bar(['Quiet (0)','Active (1)'], counts.values, color=['#2ecc71','#e74c3c'], alpha=0.8, edgecolor='black')
ax.set_title('Aurora Activity Distribution (Train)')
for i,v in enumerate(counts.values):
    ax.text(i, v + max(counts.values)*0.01, f'{v}\n({v/len(train_df)*100:.1f}%)', ha='center')

ax = axes[0,1]
missing = train_df[feature_cols].isnull().sum().sort_values(ascending=False).head(15)
if missing.sum() > 0:
    ax.barh(range(len(missing)), missing.values, color='coral', edgecolor='black')
    ax.set_yticks(range(len(missing))); ax.set_yticklabels(missing.index)
    ax.set_title('Missing Values by Feature')
else:
    ax.text(0.5,0.5,'No Missing Values ✓', ha='center', va='center')
    ax.axis('off')

ax = axes[1,0]
corrs = []
for col in feature_cols[:20]:
    if train_df[col].dtype in ['float64','int64']:
        corr = train_df[col].corr(train_df['aurora_label'])
        corrs.append((col, corr))
if corrs:
    corrs = sorted(corrs, key=lambda x: abs(x[1]), reverse=True)[:10]
    cols, vals = zip(*corrs)
    colors_corr = ['#e74c3c' if v>0 else '#3498db' for v in vals]
    ax.barh(range(len(cols)), vals, color=colors_corr, edgecolor='black')
    ax.set_yticks(range(len(cols))); ax.set_yticklabels(cols)
    ax.set_title('Top 10 Feature Correlations (abs)')
ax = axes[1,1]; ax.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '00_dataset_overview.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved 00_dataset_overview.png")

# 2) Feature distributions (KDE)
n_features = len(FEATURES_TO_PLOT)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
axes = axes.flatten()
for idx, feat in enumerate(FEATURES_TO_PLOT):
    ax = axes[idx]
    sns.kdeplot(train_df[feat].dropna(), label='Train', ax=ax, fill=True, alpha=0.5)
    sns.kdeplot(test_df[feat].dropna(), label='Test', ax=ax, fill=True, alpha=0.5)
    ax.set_title(feat)
    ax.legend(fontsize=7)
for idx in range(n_features, len(axes)):
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '01_feature_distributions.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved 01_feature_distributions.png")

# 3) Class-wise comparisons (violin + t-test)
fig, axes = plt.subplots((len(FEATURES_TO_PLOT)+1)//2, 2, figsize=(14, 4*((len(FEATURES_TO_PLOT)+1)//2)))
axes = axes.flatten()
for idx, feat in enumerate(FEATURES_TO_PLOT):
    ax = axes[idx]
    sns.violinplot(x='aurora_label', y=feat, data=train_df, ax=ax, palette={0:'#2ecc71',1:'#e74c3c'}, cut=0)
    quiet_vals = train_df[train_df['aurora_label']==0][feat].dropna()
    active_vals = train_df[train_df['aurora_label']==1][feat].dropna()
    if len(quiet_vals)>0 and len(active_vals)>0:
        tstat, pval = stats.ttest_ind(quiet_vals, active_vals, equal_var=False)
        sig = '***' if pval<0.001 else '**' if pval<0.01 else '*' if pval<0.05 else 'ns'
        ax.text(0.5, 0.95, f'p={pval:.2e} {sig}', transform=ax.transAxes, ha='center', fontsize=9, bbox=dict(facecolor='yellow', alpha=0.3))
for idx in range(len(FEATURES_TO_PLOT), len(axes)):
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '02_classwise_comparison.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved 02_classwise_comparison.png")

# 4) Time-series analysis for top 4 features
for feature in FEATURES_TO_PLOT[:4]:
    fig, axes = plt.subplots(2,1, figsize=(14,8), gridspec_kw={'height_ratios':[3,1]})
    if 'datetime' in train_df.columns:
        x_axis = pd.to_datetime(train_df['datetime'])
    else:
        x_axis = train_df.index
    window = min(100, max(10, len(train_df)//10))
    rolling_mean = train_df[feature].rolling(window=window, center=True).mean()
    rolling_std = train_df[feature].rolling(window=window, center=True).std()
    axes[0].plot(x_axis, train_df[feature], alpha=0.3, linewidth=0.5, label='Raw')
    axes[0].plot(x_axis, rolling_mean, color='blue', linewidth=2, label=f'Rolling mean (w={window})')
    axes[0].fill_between(x_axis, rolling_mean-rolling_std, rolling_mean+rolling_std, alpha=0.2, color='blue')
    active_mask = train_df['aurora_label']==1
    axes[0].scatter(x_axis[active_mask], train_df[feature][active_mask], color='red', s=8, alpha=0.6, label='Active')
    axes[0].legend()
    axes[1].fill_between(x_axis, 0, train_df['aurora_label'], color='red', alpha=0.5, step='mid')
    axes[1].set_ylim(-0.1,1.1)
    plt.tight_layout()
    fname = os.path.join(OUTPUT_DIR, f'03_timeseries_{feature}.png')
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved", fname)

# 5) Correlation matrix (top 20 features)
subset = feature_cols[:20]
if len(subset) > 1:
    corr = train_df[subset].corr()
    plt.figure(figsize=(12,10))
    sns.heatmap(corr, cmap='coolwarm', center=0)
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, '04_correlation_matrix.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved 04_correlation_matrix.png")

# 6) PCA and t-SNE visualizations
X = train_df[feature_cols].fillna(0).values
y = train_df['aurora_label'].values
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, cmap='coolwarm', s=8, alpha=0.6)
plt.title(f'PCA projection (PC1 {pca.explained_variance_ratio_[0]*100:.1f}%)')

plt.subplot(1,2,2)
sample_n = 2000 if not FAST_MODE else 1000
from sklearn.manifold import TSNE
X_sub = X[:sample_n]
y_sub = y[:sample_n]
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_sub)
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y_sub, cmap='coolwarm', s=8, alpha=0.6)
plt.title('t-SNE projection')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '06_dimensionality_reduction.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved 06_dimensionality_reduction.png")

# 7) Pairplot of top features (sampled)
top4 = FEATURES_TO_PLOT[:4]
pair_df = train_df[top4 + ['aurora_label']].sample(n=min(2000, len(train_df)), random_state=42)
g = sns.pairplot(pair_df, hue='aurora_label', diag_kind='kde', plot_kws={'alpha':0.6, 's':20})
g.fig.suptitle('Pairplot (top 4 features)', y=1.02)
plt.savefig(os.path.join(OUTPUT_DIR, '07_pairplot.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved 07_pairplot.png")

print("All analytics plots saved to:", OUTPUT_DIR)

# FINAL SUMMARY AND CONCLUSION DISPLAY

In [ ]:
# Cell 13: Summary, quick evaluation snippets, and package artifacts for download
import os, pandas as pd, joblib, glob, shutil, json
from pathlib import Path

print("Base directories:")
print(" BASE_DIR:", BASE_DIR)
print(" MODELS_DIR:", MODELS_DIR)
print(" PLOTS_DIR:", PLOTS_DIR)

# 1) List produced files
print("\nProduced files (sample):")
for pattern in [
    os.path.join(BASE_DIR, "*.csv"),
    os.path.join(MODELS_DIR, "**", "*.*"),
    os.path.join(PLOTS_DIR, "**", "*.*")
]:
    matches = glob.glob(pattern, recursive=True)
    print(f" {pattern} -> {len(matches)} files (showing up to 5):")
    for m in matches[:5]:
        print("   -", m)
print()

# 2) Show model comparison files if present
reg_comp = os.path.join(MODELS_DIR, 'regression', 'model_comparison.csv')
clf_comp = os.path.join(MODELS_DIR, 'classification', 'model_comparison_results.csv')
if os.path.exists(reg_comp):
    print("Regression comparison:")
    display(pd.read_csv(reg_comp).head())
if os.path.exists(clf_comp):
    print("Classification comparison:")
    display(pd.read_csv(clf_comp).head())

# 3) Quick: print best classification model info if available
best_info = os.path.join(MODELS_DIR, 'classification', 'best_model_info.txt')
if os.path.exists(best_info):
    print("\nBest classification model:")
    print(open(best_info).read())
else:
    print("\nNo best_model_info.txt found in classification folder.")

# 4) Quick inference demo (uses best saved classifier)
demo_out = os.path.join(BASE_DIR, 'inference_demo.csv')
try:
    # load test data
    test_path = os.path.join(BASE_DIR, 'Aurora_Test.csv')
    if os.path.exists(test_path):
        test_df = pd.read_csv(test_path)
        feat_cols = [c for c in test_df.columns if c not in ['datetime','aurora_label','time','AE','source_file']]
        sample_X = test_df[feat_cols].fillna(0).iloc[:50]
        # try to load best classifier by name from best_model_info or fallback to random_forest
        model_paths = {
            'RandomForest': os.path.join(MODELS_DIR, 'classification', 'random_forest.joblib'),
            'XGBoost': os.path.join(MODELS_DIR, 'classification', 'xgboost.joblib'),
            'LightGBM': os.path.join(MODELS_DIR, 'classification', 'lightgbm.joblib'),
            'LogisticRegression': os.path.join(MODELS_DIR, 'classification', 'logistic_regression_model.joblib')
        }
        chosen = None
        if os.path.exists(best_info):
            txt = open(best_info).read().lower()
            for k in model_paths:
                if k.lower() in txt:
                    chosen = k
                    break
        if chosen is None:
            chosen = 'RandomForest' if os.path.exists(model_paths['RandomForest']) else next((k for k,v in model_paths.items() if os.path.exists(v)), None)
        if chosen is not None and os.path.exists(model_paths[chosen]):
            print(f"\nRunning demo inference with: {chosen}")
            clf = joblib.load(model_paths[chosen])
            # if logistic regression, scale if scaler exists
            if chosen == 'LogisticRegression':
                scaler_path = os.path.join(MODELS_DIR, 'classification', 'logistic_regression_scaler.joblib')
                if os.path.exists(scaler_path):
                    scaler = joblib.load(scaler_path)
                    sample_X_s = scaler.transform(sample_X)
                else:
                    sample_X_s = sample_X.values
                preds = clf.predict_proba(sample_X_s)[:,1] if hasattr(clf, 'predict_proba') else clf.predict(sample_X_s)
            else:
                preds = clf.predict_proba(sample_X.values)[:,1] if hasattr(clf, 'predict_proba') else clf.predict(sample_X.values)
            demo_df = sample_X.reset_index(drop=True).iloc[:, :10].copy()  # show first 10 cols for brevity
            demo_df['aurora_prob'] = preds
            demo_df.to_csv(demo_out, index=False)
            print("Saved inference demo to:", demo_out)
            display(demo_df.head())
        else:
            print("No classifier found for demo inference.")
    else:
        print("Test CSV missing; cannot run demo inference.")
except Exception as e:
    print("Demo inference failed:", e)

# 5) Zip artifacts for easy download
zip_name = os.path.join(BASE_DIR, 'aurora_pipeline_artifacts.zip')
print("\nCreating zip of models & plots (this may take a moment)...")
with shutil.ZipFile(zip_name, 'w') as zf:
    # add models
    for f in glob.glob(os.path.join(MODELS_DIR, '**', '*.*'), recursive=True):
        zf.write(f, arcname=os.path.join('models', os.path.relpath(f, MODELS_DIR)))
    # add plots
    for f in glob.glob(os.path.join(PLOTS_DIR, '**', '*.*'), recursive=True):
        zf.write(f, arcname=os.path.join('plots', os.path.relpath(f, PLOTS_DIR)))
    # add CSVs
    for f in glob.glob(os.path.join(BASE_DIR, '*.csv')):
        zf.write(f, arcname=os.path.join('data', os.path.basename(f)))
print("Created zip:", zip_name)
print("You can download it from Colab using the Files sidebar or:")
print(f"  from google.colab import files\n  files.download('{zip_name}')")

# 6) Next steps summary
notes = {
    "next_steps": [
        "Inspect model metrics and confusion matrices under models/classification.",
        "Tune AE threshold or use percentile-based labeling if class imbalance is extreme.",
        "If performance is acceptable, export the best model to a serving format (ONNX / TF SavedModel) and build an inference wrapper.",
        "Set FAST_MODE=False and re-run training cells for full training.",
        "Consider logging experiments and hyperparams (MLflow or wandb)."
    ],
    "artifact_locations": {
        "models_dir": MODELS_DIR,
        "plots_dir": PLOTS_DIR,
        "data_dir": BASE_DIR,
        "zip": zip_name
    }
}
print("\nNext steps (brief):")
for i, s in enumerate(notes['next_steps'], 1):
    print(f" {i}. {s}")
print("\nArtifact locations:")
print(json.dumps(notes['artifact_locations'], indent=2))